In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# import keras
# from keras import backend as K
# print(K.tensorflow_backend._get_available_gpus())
import pandas as pd
from surprise import SVD,Dataset,accuracy, KNNWithMeans,NormalPredictor
from surprise.model_selection import train_test_split
from surprise.reader import Reader

In [2]:
data = Dataset.load_from_file('./data/ratings.csv',Reader(skip_lines=20000000,sep=','))
train,test = train_test_split(data,test_size=0.1,shuffle=True)
algo = KNNWithMeans(sim_options={'user_based':False,'name':'msd'})
algo.fit(train)
preds = algo.test(test)
accuracy.mae(preds)

Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  0.6387


0.6387323688682889

In [ ]:
data = Dataset.load_from_file('./data/ratings.csv',Reader(skip_lines=1,sep=','))
train,test = train_test_split(data,test_size=0.1,shuffle=True)
algo = SVD()
algo.fit(train)
preds = algo.test(test)
accuracy.mae(preds)

In [ ]:
algo = NormalPredictor()
algo.fit(train)
preds = algo.test(test)
accuracy.mae(preds)